In [1]:
import numpy as np
import scipy
import scipy.sparse as sp
from scipy.sparse import eye, diags, block_diag, hstack
from scipy.sparse.csgraph import reverse_cuthill_mckee
from scipy.sparse.linalg import gmres
# from scipy.sparse.csgraph import symmetrix_degree_order
import importlib
import time
import NavierStokes_Periodic_Solver as NS_Per

In [3]:
importlib.reload(NS_Per)

<module 'NavierStokes_Periodic_Solver' from '/Users/huynh/Desktop/Python/EnSF/NavierStokes/Periodic/NavierStokes_Periodic/NavierStokes_Periodic_Solver.py'>

# Reference solution using BDF2

## Setting for 1 step BE

In [5]:
data = scipy.io.loadmat('Permutation_Indices_RefSol_Per40.mat')

# Convert MATLAB arrays to NumPy arrays and cast to float64.
perS = data['perS'].astype(np.float64)
# perA = data['perA'].astype(np.float64)

In [7]:
perS -= 1
# perA -= 1

In [9]:
# Assume the following helper functions are defined:
# -------------------------------
# Domain discretization and time setup
xa = 0 
xb = 1
ya = 0 
yb = 1 
T = 1

mu = 0.001
# n = 8; Nx = 32*n; Ny = Nx; Nt = 8*n;

theta = 5
Nx = 40
Ny = Nx
Nt = T*600

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny
# Create grid points: MATLAB: x = xa:hx:xb, y = ya:hy:yb
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)
xmid = NS_Per.avg(x)
ymid = NS_Per.avg(y)
dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

alpha_BE = 1 / dt   # alpha*u - mu*Delta(u) + grad(p) = f
opt_UgradU = 1   # 1: original, 2: MIT (not good)
opt = 2
# -------------------------------
# Construct matrices A and B

# Sizes:
sU = (Nx) * Ny       # for U-velocity unknowns
sV = sU       # for V-velocity unknowns
sP = sU             # for pressure


# --- Build matrix A ---
A0  = NS_Per.DiscreteLaplace(Nx,hx)
B0  = NS_Per.DiscreteLaplace(Ny,hy)

A_u = alpha_BE * sp.eye(sU) - mu * (sp.kron(sp.eye(Ny), A0) + sp.kron(B0, sp.eye(Nx)))
A_v = A_u.copy()

A_BE = block_diag((A_u, A_v), format='csr')

# # --- Construct matrix B ---

A1 = NS_Per.DiscreteGrad(Nx,hx)         # P_x = A1*P
B1 = NS_Per.DiscreteGrad(Ny,hy)         # P_y = P*B1'

B2 = sp.kron(sp.eye(Ny), A1.T)
B3 = sp.kron(B1.T, sp.eye(Nx))
B = sp.hstack([B2, B3], format='csr')

B = B[1:, :]
Bt = B.transpose().tocsr()

# # --- Prepare matrices for the pressure correction ---
dA_BE = A_BE.diagonal()
D_BE = diags(dA_BE, 0, shape=A_BE.shape, format='csr')
E_BE = D_BE - A_BE
Di_BE = diags(1.0 / dA_BE, 0, shape=A_BE.shape, format='csr')
S_BE = B.dot(Di_BE.dot(Bt))
# # perS = reverse_cuthill_mckee(S)
# # S_perm = S[perS, :][:, perS].toarray()

rowsS, colsS = np.meshgrid(perS, perS)
S_perm = S_BE[rowsS, colsS].toarray()
SS_BE = np.linalg.cholesky(S_perm).T
SS_BEt = SS_BE.T
DiE_BE = Di_BE.dot(E_BE)
BDiE_BE = B.dot(DiE_BE)
DiB_BEt = Di_BE.dot(Bt)

## Create mesh
Yu, Xu = np.meshgrid(ymid, x[0:-1], indexing='xy')

Yv, Xv = np.meshgrid(y[0:-1], xmid, indexing='xy')

Yp, Xp = np.meshgrid(ymid, xmid, indexing='xy')

# Initialize velocity fields using your exact solution functions.
U0 = NS_Per.u_init(Xu, Yu, opt)  # dimensions should match (len(x[1:-1]) x len(ymid))
V0 = NS_Per.v_init(Xv, Yv, opt)

# Initialize pressure-related quantities.
q = 1

q_batch = np.full((2,), q)
# qq = np.zeros(Nt+1)
# qq[0] = q

# egy = np.zeros(Nt + 1)
# egy_theta = egy.copy()
# egy[0] = 0.5 * hx * hy * (NS_Per.inner(U0, U0) + NS_Per.inner(V0, V0))
# egy_theta[0] = egy[0]+theta*(q**2-1)

In [11]:
perSBE_new = perS.astype(int)

perSBE_new = np.squeeze(perSBE_new, axis=0)

## Setting for BDF2

In [13]:
data = scipy.io.loadmat('Permutation_Indices_RefSol_BDF2Per40.mat')

# Convert MATLAB arrays to NumPy arrays and cast to float64.
perBDFS = data['perS'].astype(np.float64)

In [15]:
perBDFS -= 1

In [17]:
# Assume the following helper functions are defined:
# -------------------------------
# Domain discretization and time setup
xa = 0 
xb = 1
ya = 0 
yb = 1 
T = 1

mu = 0.001
# n = 8; Nx = 32*n; Ny = Nx; Nt = 8*n;

theta = 5
Nx = 40
Ny = Nx
Nt = T*600

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny
# Create grid points: MATLAB: x = xa:hx:xb, y = ya:hy:yb
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)
xmid = NS_Per.avg(x)
ymid = NS_Per.avg(y)
dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

alpha = 1.5 / dt   # alpha*u - mu*Delta(u) + grad(p) = f
opt_UgradU = 1   # 1: original, 2: MIT (not good)
opt = 2
# -------------------------------
# Construct matrices A and B

# Sizes:
sU = (Nx) * Ny       # for U-velocity unknowns
sV = sU       # for V-velocity unknowns
sP = sU             # for pressure


# --- Build matrix A ---
A0  = NS_Per.DiscreteLaplace(Nx,hx)
B0  = NS_Per.DiscreteLaplace(Ny,hy)

A_u = alpha * sp.eye(sU) - mu * (sp.kron(sp.eye(Ny), A0) + sp.kron(B0, sp.eye(Nx)))
A_v = A_u.copy()

A = block_diag((A_u, A_v), format='csr')

# # --- Construct matrix B ---

A1 = NS_Per.DiscreteGrad(Nx,hx)         # P_x = A1*P
B1 = NS_Per.DiscreteGrad(Ny,hy)         # P_y = P*B1'

B2 = sp.kron(sp.eye(Ny), A1.T)
B3 = sp.kron(B1.T, sp.eye(Nx))
B = sp.hstack([B2, B3], format='csr')

B = B[1:, :]
Bt = B.transpose().tocsr()

# # --- Prepare matrices for the pressure correction ---
dA = A.diagonal()
D = diags(dA, 0, shape=A.shape, format='csr')
E = D - A
Di = diags(1.0 / dA, 0, shape=A.shape, format='csr')
S = B.dot(Di.dot(Bt))
# # perS = reverse_cuthill_mckee(S)
# # S_perm = S[perS, :][:, perS].toarray()

rowsS, colsS = np.meshgrid(perBDFS, perBDFS)
S_perm = S[rowsS, colsS].toarray()
SS = np.linalg.cholesky(S_perm).T
SSt = SS.T
DiE = Di.dot(E)
BDiE = B.dot(DiE)
DiBt = Di.dot(Bt)

## Create mesh
Yu, Xu = np.meshgrid(ymid, x[0:-1], indexing='xy')

Yv, Xv = np.meshgrid(y[0:-1], xmid, indexing='xy')

Yp, Xp = np.meshgrid(ymid, xmid, indexing='xy')

# Initialize velocity fields using your exact solution functions.
U0 = NS_Per.u_init(Xu, Yu, opt)  # dimensions should match (len(x[1:-1]) x len(ymid))
V0 = NS_Per.v_init(Xv, Yv, opt)

# Initialize pressure-related quantities.
q = 1

# q_batch = np.full((2,), q)
# qq = np.zeros(Nt+1)
# qq[0] = q

# egy = np.zeros(Nt + 1)
# egy_theta = egy.copy()
# egy[0] = 0.5 * hx * hy * (NS_Per.inner(U0, U0) + NS_Per.inner(V0, V0))
# egy_theta[0] = egy[0]+theta*(q**2-1)

In [19]:
perBDFS_new = perBDFS.astype(int)

perBDFS_new = np.squeeze(perBDFS_new, axis=0)

In [21]:
mU0, nU0 = U0.shape
mV0, nV0 = V0.shape
mP0, nP0 = Xp.shape

In [23]:
All_U = np.zeros((Nt+1, mU0*nU0))
All_V = np.zeros((Nt+1, mV0*nV0))
All_P = np.zeros((Nt+1, mP0*nP0))
All_q = np.zeros((Nt+1, 1))

In [25]:
All_U[0, :] += np.reshape(U0, mU0*nU0, order='F')
All_V[0, :] += np.reshape(V0, mV0*nV0, order='F')
All_q[0, :] += 1

In [523]:
print(All_U.shape)

(601, 1600)


### 1 step BE

In [354]:
q0 = 1
U = U0.copy()
V = V0.copy()
## nt = 1 here
for k in range(1):
    print(k)
    U_new, V_new, P_new, q_new, egy_new, egy_theta_new,_ = \
            NS_Per.NS_BE_1step_Periodic(hx, hy, dt, TT[k+1], U, V, q0, Xu, Yu, Xv, Yv, mu, theta, opt, opt_UgradU,\
                                        DiE_BE, BDiE_BE, DiB_BEt, Di_BE, B, Bt, perSBE_new, SS_BE, SS_BEt, Nx, Ny,\
                                        sU, alpha_BE, A1, B1)
    
#     print(q_new)
#     qq[k+1] = q_new
    U = U_new.copy()
    V = V_new.copy()
    P = P_new.copy()
    q0 = q_new.copy()
    
    All_U[1, :] += np.reshape(U, mU0*nU0, order='F')
    All_V[1, :] += np.reshape(V, mV0*nV0, order='F')
    All_P[1, :] += np.reshape(P, mP0*nP0, order='F')
    All_q[1, :] += q_new

0


In [356]:
U_old_old = U0.copy()
V_old_old = V0.copy()
U_old = U_new.copy()
V_old = V_new.copy()
q_old = q_new.copy()

In [358]:
# qq[1] = q_old
q_old_old = q

In [651]:
for kk in range(1, Nt):
    print(kk)
    U_new, V_new, P_new, q_new, _= \
                NS_Per.NS_BDF2_1step_periodic(hx, hy, dt, TT[kk+1], U_old_old, V_old_old, U_old, V_old,\
                                              q_old, q_old_old, Xu, Yu, Xv, Yv, mu, theta, opt, opt_UgradU,\
                                              DiE, BDiE, DiBt, Di, B, Bt, perBDFS_new, SS, SSt, Nx, Ny, sU, \
                                              alpha, A1, B1)
        
    ## Update
    U_old_old = U_old.copy()
    V_old_old = V_old.copy()
    
    U_old = U_new.copy()
    V_old = V_new.copy()
    
    q_old_old = q_old
    q_old = q_new
    
#     qq[kk+1] = q_new
    
    All_U[kk+1, :] += np.reshape(U_new, mU0*nU0, order='F')
    All_V[kk+1, :] += np.reshape(V_new, mV0*nV0, order='F')
    All_P[kk+1, :] += np.reshape(P_new, mP0*nP0, order='F')
    All_q[kk+1, :] += q_new

1
2
3
4
5
6
7
8
9
10


KeyboardInterrupt: 

In [362]:
U_Ref = All_U.copy()
V_Ref = All_V.copy()
P_Ref = All_P.copy()
q_Ref = All_q.copy()

In [364]:
scipy.io.savemat('TestRefSol_BDF2_Periodic_v2.mat', \
                 {'U_Py':U_Ref, 'V_Py':V_Ref, 'P_Py':P_Ref, 'q_Py':q_Ref})

# Performing EnSF with BE

In [27]:
def cond_alpha(t):
    # conditional information
    # alpha_t(0) = 1
    # alpha_t(1) = esp_alpha \approx 0
    return 1 - (1-eps_alpha)*t


def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def f(t):
    # f=d_(log_alpha)/dt
    alpha_t = cond_alpha(t)
    f_t = -(1-eps_alpha) / alpha_t
    return f_t

def g_sq(t):
    # g = d(sigma_t^2)/dt -2f sigma_t^2
    d_sigma_sq_dt = 1
    g2 = d_sigma_sq_dt - 2*f(t)*cond_sigma_sq(t)
    return g2

def g(t):
    return np.sqrt(g_sq(t))

def reverse_SDE(x0, time_steps, C, score_likelihood=None, drift_fun=f, \
                diffuse_fun=g, alpha_fun=cond_alpha, sigma2_fun=cond_sigma_sq, save_path=False):
    # x_T: sample from standard Gaussian
    # x_0: target distribution to sample from

    # reverse SDE sampling process
    # N1 = x_T.shape[0]
    # N2 = x0.shape[0]
    # d = x_T.shape[1]

    # Generate the time mesh
    dt = 1.0/time_steps

    # Initialization
    xt = np.random.randn(x0.shape[0], x0.shape[1])
    t = 1.0
    
    path_all = []
    t_vec = []
    
    # define storage
    if save_path:
        path_all.append(xt.copy())
        t_vec.append(t)
    
    # forward Euler sampling
    for i in range(time_steps):
        # prior score evaluation
        alpha_t = alpha_fun(t)
        sigma2_t = sigma2_fun(t)


        # Evaluate the diffusion term
        diffuse = diffuse_fun(t)

        # Evaluate the drift term
        # drift = drift_fun(t)*xt - diffuse**2 * score_eval

        # Update
        if score_likelihood is not None:
#             zt = score_likelihood(xt, t)
#             print(zt.size())
            xt += -dt*(drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t)-\
                       diffuse**2*score_likelihood(xt, t, C)) +np.sqrt(dt)*diffuse*np.random.randn(*xt.shape)
    
        else:
            xt += -dt*(drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t))+np.sqrt(dt)*diffuse*np.random.randn(*xt.shape)
        
#         xt = xt.to(torch.float64)
        # Store the state in the path
        if save_path:
            path_all.append(xt.copy())
            t_vec.append(t)

        # update time
        t = t - dt

    if save_path:
        return path_all, t_vec
    else:
        return xt

In [30]:
# mU, nU = U_Ref.shape
# mV, nV = V_Ref.shape
# mP, nP = P_Ref.shape

In [29]:
Size_U = mU0*nU0
Size_V = mV0*nV0
Size_P = mP0*nP0

In [53]:
# ## 100% arctangent observation
indices_U = np.random.permutation(Size_U)[:int(1 * Size_U)]
indices_V = np.random.permutation(Size_V)[:int(1 * Size_V)] + Size_U
indices_P = np.random.permutation(Size_P)[:int(1 * Size_P)] + Size_U + Size_V
indices_q = np.array(Size_U+Size_V+Size_P)


## 70% arctangent observation
# indices_U = np.random.permutation(Size_U)[:int(0.7 * Size_U)]
# indices_V = np.random.permutation(Size_V)[:int(0.7 * Size_V)] + Size_U
# indices_P = np.random.permutation(Size_P)[:int(0.7 * Size_P)] + Size_U + Size_V
# indices_q = np.array(Size_U+Size_V+Size_P)


## 7% arctangent observation
# indices_U = np.random.permutation(Size_U)[:int(0.07* Size_U)]
# indices_V = np.random.permutation(Size_V)[:int(0.07* Size_V)] + Size_U
# indices_P = np.random.permutation(Size_P)[:int(0.07* Size_P)] + Size_U + Size_V
# indices_q = np.array(Size_U+Size_V+Size_P)
num_indices= indices_U.size+indices_V.size+indices_P.size+1

# Combine and sort the selected indices
spa_indices = np.sort(np.concatenate([indices_U, indices_V, indices_P, indices_q.reshape(1)]))
print(num_indices)
print(indices_q)

337
4800


In [55]:
data1 = scipy.io.loadmat('TestRefSol_BDF2_Periodic_v2.mat')
U_Ref = data1['U_Py']
V_Ref = data1['V_Py']
P_Ref = data1['P_Py']
q_Ref = data1['q_Py']

In [77]:
# ntEnSF = 50
ntEnSF = 100
t0 = 0
filtering_steps = ntEnSF
timeTrue = np.linspace(0, 1, Nt+1)
tEnSF = np.linspace(0, 1, filtering_steps+1)
indices_time = np.searchsorted(timeTrue, tEnSF, side='left')


state_ref = np.concatenate((U_Ref, V_Ref, P_Ref, q_Ref), axis=1)   

print(state_ref.shape)
state_timeextract = state_ref[indices_time, :].copy()

state_EnSF = state_timeextract[:, spa_indices].copy()

dtEnSF = (T - t0) / ntEnSF
obs_sigma = 0.1

eps_alpha = 0.05

# ensemble size
ensemble_size = 80
ensemble_true = 1
# forward Euler step
euler_steps = 400
def g_tau(t):
    return 1-t

U0_state = 2*np.random.randn(ensemble_size, Size_U)
V0_state = 2*np.random.randn(ensemble_size, Size_V)
P0_state = 2*np.random.randn(ensemble_size, Size_P)

UV_state = np.concatenate((U0_state, V0_state, P0_state, np.full((ensemble_size, 1), 1)), axis=1)

n_dim = Size_U+Size_V+Size_P+1
rmse_all = []
obs_save = []
est_save = np.zeros((filtering_steps+1, n_dim))
est_save[[0], :] += np.mean(UV_state, axis=0)

(601, 4801)


In [79]:
## Noise omega_1
SDE_Sigma_U = 0.001
SDE_Sigma_V = 0.001
SDE_Sigma_P = 0.001

## Noise omega_2
# SDE_Sigma_U = 0.1
# SDE_Sigma_V = 0.1
# SDE_Sigma_P = 0.1

In [81]:
data1 = scipy.io.loadmat('Permutation_Indices_EnSF_Per40_T100.mat')
# data1 = scipy.io.loadmat('Permutation_Indices_EnSF_Per40.mat')
# Convert MATLAB arrays to NumPy arrays and cast to float64.
perS_EnSF = data1['perS'].astype(np.float64)

In [83]:
perS_EnSF -= 1

In [85]:
# Assume the following helper functions are defined:
# -------------------------------
# Domain discretization and time setup
xa = 0 
xb = 1
ya = 0 
yb = 1 
T = 1

mu = 0.001
# n = 8; Nx = 32*n; Ny = Nx; Nt = 8*n;

theta = 5
Nx = 40
Ny = Nx
# Nt = T*50

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny
# Create grid points: MATLAB: x = xa:hx:xb, y = ya:hy:yb
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)
xmid = NS_Per.avg(x)
ymid = NS_Per.avg(y)
# dt = T / Nt
TTEnSF = np.arange(0, T + dtEnSF/2, dtEnSF)

alpha_EnSFBE = 1 / dtEnSF   # alpha*u - mu*Delta(u) + grad(p) = f
opt_UgradU = 1   # 1: original, 2: MIT (not good)
opt = 2
# -------------------------------
# Construct matrices A and B

# Sizes:
sU = (Nx) * Ny       # for U-velocity unknowns
sV = sU       # for V-velocity unknowns
sP = sU             # for pressure


# --- Build matrix A ---
A0  = NS_Per.DiscreteLaplace(Nx,hx)
B0  = NS_Per.DiscreteLaplace(Ny,hy)

A_u = alpha_EnSFBE * sp.eye(sU) - mu * (sp.kron(sp.eye(Ny), A0) + sp.kron(B0, sp.eye(Nx)))
A_v = A_u.copy()

A_EnSFBE = block_diag((A_u, A_v), format='csr')

# # --- Construct matrix B ---

A1 = NS_Per.DiscreteGrad(Nx,hx)         # P_x = A1*P
B1 = NS_Per.DiscreteGrad(Ny,hy)         # P_y = P*B1'

B2 = sp.kron(sp.eye(Ny), A1.T)
B3 = sp.kron(B1.T, sp.eye(Nx))
B = sp.hstack([B2, B3], format='csr')

B = B[1:, :]
Bt = B.transpose().tocsr()

# # --- Prepare matrices for the pressure correction ---
dA_EnSFBE = A_EnSFBE.diagonal()
D_EnSFBE = diags(dA_EnSFBE, 0, shape=A_EnSFBE.shape, format='csr')
E_EnSFBE = D_EnSFBE - A_EnSFBE
Di_EnSFBE = diags(1.0 / dA_EnSFBE, 0, shape=A_EnSFBE.shape, format='csr')
S_EnSFBE = B.dot(Di_EnSFBE.dot(Bt))
# # perS = reverse_cuthill_mckee(S)
# # S_perm = S[perS, :][:, perS].toarray()

rowsS, colsS = np.meshgrid(perS_EnSF, perS_EnSF)
S_perm = S_EnSFBE[rowsS, colsS].toarray()
SS_EnSFBE = np.linalg.cholesky(S_perm).T
SS_EnSFBEt = SS_EnSFBE.T
DiE_EnSFBE = Di_EnSFBE.dot(E_EnSFBE)
BDiE_EnSFBE = B.dot(DiE_EnSFBE)
DiB_EnSFBEt = Di_EnSFBE.dot(Bt)

## Create mesh
Yu, Xu = np.meshgrid(ymid, x[0:-1], indexing='xy')

Yv, Xv = np.meshgrid(y[0:-1], xmid, indexing='xy')

Yp, Xp = np.meshgrid(ymid, xmid, indexing='xy')

# Initialize velocity fields using your exact solution functions.
# U0 = NS_Per.u_init(Xu, Yu, opt)  # dimensions should match (len(x[1:-1]) x len(ymid))
# V0 = NS_Per.v_init(Xv, Yv, opt)

# Initialize pressure-related quantities.
q = 1

q_batch = np.full((ensemble_size,), q)
# qq = np.zeros(Nt+1)
# qq[0] = q

# egy = np.zeros(Nt + 1)
# egy_theta = egy.copy()
# egy[0] = 0.5 * hx * hy * (NS_Per.inner(U0, U0) + NS_Per.inner(V0, V0))
# egy_theta[0] = egy[0]+theta*(q**2-1)

In [87]:
perS_EnSFnew = perS_EnSF.astype(int)

perS_EnSFnew = np.squeeze(perS_EnSFnew, axis=0)

In [89]:
for i in range(filtering_steps):
    print(f'step={i}:')
    t1 = time.time()    
    
#     obs = state_EnSF[[i+1], :].copy()
    state_scale = state_EnSF[[i+1], :].copy()
    
    indob_scale0 = np.nonzero(((-1e-1 <= state_scale) & (state_scale < -1e-2)) |
                              ((1e-2 <= state_scale) & (state_scale < 1e-1)))[1]
    
    indob_scale1 = np.nonzero(((-1e-2 <= state_scale) & (state_scale < -1e-3)) |
                              ((1e-3 <= state_scale) & (state_scale < 1e-2)))[1]
    
    indob_scale2 = np.nonzero(((-1e-3 <= state_scale) & (state_scale < -1e-4)) |
                              ((1e-4 <= state_scale) & (state_scale < 1e-3)))[1]

    indob_scale3 = np.nonzero(((-1e-4 <= state_scale) & (state_scale < -1e-5)) |
                              ((1e-5 <= state_scale) & (state_scale < 1e-4)))[1]

    indob_scale4 = np.nonzero(((-1e-5 <= state_scale) & (state_scale < -1e-6)) |
                              ((1e-6 <= state_scale) & (state_scale < 1e-5)))[1]

    indob_scale5 = np.nonzero(((-1e-12 <= state_scale) & (state_scale < -1e-13)) |
                              ((1e-13 <= state_scale) & (state_scale < 1e-12)))[1]

    indob_scale6 = np.nonzero(((-1e-13 <= state_scale) & (state_scale < -1e-14)) |
                              ((1e-14 <= state_scale) & (state_scale < 1e-13)))[1]
    
    indob_scale7 = np.nonzero(((-1e-14 <= state_scale) & (state_scale < -1e-15)) |
                              ((1e-15 <= state_scale) & (state_scale < 1e-14)))[1]
    
    indob_scale8 = np.nonzero(((-1e-15 <= state_scale) & (state_scale < -1e-16)) |
                              ((1e-16 <= state_scale) & (state_scale < 1e-15)))[1]

    indob_scale9 = np.nonzero(((-1e-16 <= state_scale) & (state_scale < -1e-17)) |
                              ((1e-17 <= state_scale) & (state_scale < 1e-16)))[1]
    
    # state_scale[:, indob_scale1] *= 5e1
    state_scale[:, indob_scale1] *= 1e2
    state_scale[:, indob_scale2] *= 1e3
    state_scale[:, indob_scale3] *= 1e4
    state_scale[:, indob_scale4] *= 1e5
    state_scale[:, indob_scale5] *= 1e12
    state_scale[:, indob_scale6] *= 1e13
    state_scale[:, indob_scale7] *= 1e14
    state_scale[:, indob_scale8] *= 1e15
    state_scale[:, indob_scale9] *= 1e16
    
    obs = np.arctan(state_scale.copy())
    obs += np.random.randn(*state_EnSF[[i+1], :].shape) * obs_sigma

    def score_likelihood(xt, t, C):
        # obs: (d)
        # xt: (ensemble, d)
#         A = -(xt - obs) / obs_sigma**2
#         score_x = A.copy()
#         score_x[:, idA_sub] =\
#             -(np.arctan(xt[:, idA_sub]) - obs[:, idA_sub]) / obs_sigma**2 * (1. / (1 + xt[:, idA_sub]**2))
        score_x = -(np.arctan(xt) - obs)/obs_sigma**2 * (1./(1+xt**2))
        tau = g_tau(t)
        return tau * score_x / C
       
    U_stack = (U0_state.T).copy()
    V_stack = (V0_state.T).copy()
    
    U_stack = U_stack.reshape(mU0, nU0, ensemble_size, order = 'F')
    V_stack = V_stack.reshape(mV0, nV0, ensemble_size, order = 'F')
    
    U_new, V_new, P_new, q_new, egy_new, egy_theta_new,_ = \
            NS_Per.NS_BE_1step_Periodic_Vectorized(hx, hy, dtEnSF, TTEnSF[i+1], U_stack, V_stack, q_batch, Xu, Yu, Xv, Yv,\
                                                   mu, theta, opt, opt_UgradU, DiE_EnSFBE, BDiE_EnSFBE, DiB_EnSFBEt,\
                                                   Di_EnSFBE, B, Bt, perS_EnSFnew, SS_EnSFBE, SS_EnSFBEt, Nx, Ny, sU,\
                                                   alpha_EnSFBE, A1, B1)
    
    U_new_reshape = U_new.reshape(mU0*nU0, ensemble_size, order ='F')  
    V_new_reshape = V_new.reshape(mV0*nV0, ensemble_size, order ='F')
    P_new_reshape = P_new.reshape(mP0*nP0, ensemble_size, order ='F')
    q_new_reshape = q_new.reshape(1, ensemble_size)
    
    # q_batch = q_new.copy()
    x_state = np.concatenate((U_new_reshape, V_new_reshape, P_new_reshape, q_new_reshape))
    
    noiseU = np.sqrt(dtEnSF) * SDE_Sigma_U * np.random.randn(*U_new_reshape.shape)
    noiseV = np.sqrt(dtEnSF) * SDE_Sigma_V * np.random.randn(*V_new_reshape.shape)
    noiseP = np.sqrt(dtEnSF) * SDE_Sigma_P * np.random.randn(*P_new_reshape.shape)

    noise = np.concatenate((noiseU, noiseV, noiseP, np.zeros((1, ensemble_size))))
    
    x_state += noise
    x_state = x_state.T
    
    x0_EnSF = x_state[:, spa_indices].copy()

    for l in range(7):      
        indx_scale0 = np.argwhere(((-1e-1<=x0_EnSF) & (x0_EnSF<-1e-2)) | ((1e-2<=x0_EnSF) & (x0_EnSF<1e-1)))
        indx_scale1 = np.argwhere(((-1e-2<=x0_EnSF) & (x0_EnSF<-1e-3)) | ((1e-3<=x0_EnSF) & (x0_EnSF<1e-2)))
        indx_scale2 = np.argwhere(((-1e-3<=x0_EnSF) & (x0_EnSF<-1e-4)) | ((1e-4<=x0_EnSF) & (x0_EnSF<1e-3)))
        indx_scale3 = np.argwhere(((-1e-4<=x0_EnSF) & (x0_EnSF<-1e-5)) | ((1e-5<=x0_EnSF) & (x0_EnSF<1e-4)))
        indx_scale4 = np.argwhere(((-1e-5<=x0_EnSF) & (x0_EnSF<-1e-6)) | ((1e-6<=x0_EnSF) & (x0_EnSF<1e-5)))
        indx_scale5 = np.argwhere(((-1e-12<=x0_EnSF) & (x0_EnSF<-1e-13)) | ((1e-13<=x0_EnSF) & (x0_EnSF<1e-12)))
        indx_scale6 = np.argwhere(((-1e-13<=x0_EnSF) & (x0_EnSF<-1e-14)) | ((1e-14<=x0_EnSF) & (x0_EnSF<1e-13)))
        indx_scale7 = np.argwhere(((-1e-14<=x0_EnSF) & (x0_EnSF<-1e-15)) | ((1e-15<=x0_EnSF) & (x0_EnSF<1e-14)))
        indx_scale8 = np.argwhere(((-1e-15<=x0_EnSF) & (x0_EnSF<-1e-16)) | ((1e-16<=x0_EnSF) & (x0_EnSF<1e-15)))
        indx_scale9 = np.argwhere(((-1e-16<=x0_EnSF) & (x0_EnSF<-1e-17)) | ((1e-17<=x0_EnSF) & (x0_EnSF<1e-16)))
        
        # # # x0_EnSF[indx_scale1[:, 0], indx_scale1[:, 1]] *= 5e1
        x0_EnSF[indx_scale1[:, 0], indx_scale1[:, 1]] *= 1e2
        x0_EnSF[indx_scale2[:, 0], indx_scale2[:, 1]] *= 1e3
        x0_EnSF[indx_scale3[:, 0], indx_scale3[:, 1]] *= 1e4
        x0_EnSF[indx_scale4[:, 0], indx_scale4[:, 1]] *= 1e5
        x0_EnSF[indx_scale5[:, 0], indx_scale5[:, 1]] *= 1e12
        x0_EnSF[indx_scale6[:, 0], indx_scale6[:, 1]] *= 1e13
        x0_EnSF[indx_scale7[:, 0], indx_scale7[:, 1]] *= 1e14
        x0_EnSF[indx_scale8[:, 0], indx_scale8[:, 1]] *= 1e15
        x0_EnSF[indx_scale9[:, 0], indx_scale9[:, 1]] *= 1e16

        sln_bar = reverse_SDE(x0=x0_EnSF.copy(), time_steps=euler_steps,  C=1, score_likelihood=score_likelihood)

        ## v1a
        sln_bar[:, indob_scale1] /= 1e2
        sln_bar[:, indob_scale2] /= 1e3
        sln_bar[:, indob_scale3] /= 1e4
        sln_bar[:, indob_scale4] /= 1e5
        sln_bar[:, indob_scale5] /= 1e12
        sln_bar[:, indob_scale6] /= 1e13
        sln_bar[:, indob_scale7] /= 1e14
        sln_bar[:, indob_scale8] /= 1e15
        sln_bar[:, indob_scale9] /= 1e16

        # print(sln_bar[:, Size_U+Size_V+np.arange(0, Size_P)])
        x0_EnSF = sln_bar.copy()

    x_state[:, spa_indices] = sln_bar.copy()
    x_state[:, Size_U+Size_V+np.arange(0, Size_P)] = np.clip(x_state[:, Size_U+Size_V+np.arange(0, Size_P)], -0.6, 0.6)
    x_state[:, -1] = np.clip(x_state[:, -1], 0.95, 1.05)
    
    # print(x_state[:, Size_U+Size_V+np.arange(0, Size_P)])
    U0_state = x_state[:, :Size_U].copy()
    V0_state = x_state[:, Size_U+np.arange(0, Size_V)].copy()
    q_batch = x_state[:, -1].copy()
    print(q_Ref[i+1])
    # print(q_batch)
    est_save[[i+1], :] += np.mean(x_state, axis=0)
    print(est_save[i+1, -1])
    # get rmse
    rmse_temp = np.sqrt(np.mean((est_save[[i+1], :] - state_timeextract[[i+1], :])**2))

    # get time
    t2 = time.time()
    print(f'\t RMSE = {rmse_temp:.4f}')
    print(f'\t time = {t2 - t1:.4f}')

    # save information
    rmse_all.append(rmse_temp)

    # check divergence
    if rmse_temp > 1000:
        print('diverge!')
        break

step=0:
[1.]
1.0151302029974434
	 RMSE = 0.4456
	 time = 154.6737
step=1:
[1.]
0.9968167696544036
	 RMSE = 0.4362
	 time = 76.4942
step=2:
[1.]
0.9957854996840207
	 RMSE = 0.4284
	 time = 130.3630
step=3:
[1.]
0.9835400029080466
	 RMSE = 0.4231
	 time = 118.0575
step=4:
[1.]
0.9934438875997771
	 RMSE = 0.4191
	 time = 113.7985
step=5:
[1.]
1.0027764301298006
	 RMSE = 0.4159
	 time = 75.1071
step=6:
[1.]
1.0019275615327203
	 RMSE = 0.4122
	 time = 77.9060
step=7:
[1.]
0.9822761867551885
	 RMSE = 0.4081
	 time = 82.1468
step=8:
[1.]
0.981360520692369
	 RMSE = 0.4064
	 time = 83.3445
step=9:
[1.]
1.0090512392163073
	 RMSE = 0.4055
	 time = 77.0155
step=10:
[1.]
0.9561559692172455
	 RMSE = 0.4085
	 time = 75.8171
step=11:
[1.]
0.9703395372418353
	 RMSE = 0.4137
	 time = 73.2535
step=12:
[1.]
0.9641100616190883
	 RMSE = 0.4250
	 time = 77.6833
step=13:
[1.]
0.9524066246188122
	 RMSE = 0.4440
	 time = 101.6112
step=14:
[1.]
0.9569715308705782
	 RMSE = 0.4629
	 time = 77.4552
step=15:
[1.]
0.

In [91]:
U_EnSF = est_save[:, :Size_U]
V_EnSF = est_save[:, Size_U+np.arange(0, Size_V)]
P_EnSF = est_save[:, Size_U+Size_V+np.arange(0, Size_P)]
q_EnSF = est_save[:, -1]

In [93]:
scipy.io.savemat('ResultEnSF_Periodic_T100_10Obs_noise0001_v1.mat', {'U_EnSF':U_EnSF, 'V_EnSF':V_EnSF, 'P_EnSF':P_EnSF, \
                                                                     'q_EnSF': q_EnSF, 'rmse': rmse_all})
# scipy.io.savemat('ResultEnSF_Periodic_T100_10Obs_noise01_v1.mat', {'U_EnSF':U_EnSF, 'V_EnSF':V_EnSF, 'P_EnSF':P_EnSF, \
#                                                                      'q_EnSF': q_EnSF, 'rmse': rmse_all})

# scipy.io.savemat('ResultEnSF_Periodic_T100_7Obs_noise0001_v1.mat', {'U_EnSF':U_EnSF, 'V_EnSF':V_EnSF, 'P_EnSF':P_EnSF, \
#                                                'q_EnSF': q_EnSF, 'rmse': rmse_all})
# scipy.io.savemat('ResultEnSF_Periodic_T100_7Obs_noise01_v1.mat', {'U_EnSF':U_EnSF, 'V_EnSF':V_EnSF, 'P_EnSF':P_EnSF, \
#                                                'q_EnSF': q_EnSF, 'rmse': rmse_all})